# Confirmation of DAVIS dataset
    Just to make sure DAVIS is outputing the correct data
   

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

from vos.datasets.DAVIS import DAVIS_2017_TrainVal
from vos.datasets.video_synth import VideoSynthDataset
from vos.datasets.COCO import COCO

from vos.utils.helpers import stack_images, stack_masks

NOTE: The following two dataset, only need to run one

In [7]:
# davisroot = "/p300/videoObjSeg_dataset/DAVIS-2017-trainval-480p"
# videod = DAVIS_2017_TrainVal(davisroot, mode= "train")

cocoroot = "/p300/videoObjSeg_dataset/COCO-2017-train"
videod = VideoSynthDataset(
    COCO(cocoroot, mode= "train", max_n_objects= 8),
    n_frames= 3,
    affine_kwargs= dict(
        angle_max= 5.,
        translate_max= 10.,
        scale_max= 0.05, # NOTE: this is the exponent of e
        shear_max= 0.
    ),
)

d_len = len(videod)
data = videod[np.random.choice(d_len, 1)[0]]
for k, v in data.items():
    print(k, ": ", v.shape)

loading annotations into memory...
Done (t=45.51s)
creating index...
index created!
mask :  torch.Size([3, 9, 384, 384])
bboxs :  (8, 4)
n_objects :  torch.Size([])
video :  torch.Size([3, 3, 384, 384])


In [ ]:
s_images = stack_images(np.expand_dims(data["video"].numpy(), axis=0))
s_mask = stack_masks(np.expand_dims(data["mask"].numpy(), axis=0)[0])
no = data["n_objects"]
print(no, s_mask.shape)

plt.imshow(s_images[0].transpose(1,2,0))
plt.figure()
plt.imshow(s_mask[0,0]*255)

tensor(2) (1, 1, 1152, 3456)


## Conclusion
    - DAVIS dataset outputs proper masks whose valid information are at (0:n_objects+1) channel
    - Vieo Synthesize Dataset wrapper has issue, need further investigation
    
## The following code only use videod object from previous codes.

In [ ]:
import torch
d_len = len(videod)
vdata = videod[np.random.choice(d_len, 1)[0]]
for k, v in vdata.items():
    print(k, ": ", v.shape)

In [ ]:
# Then reproducing the video synthesize dataset getitem procedure
i_len = len(videod.dataset)
idata = videod.dataset[np.random.choice(i_len, 1)[0]]
image = idata["image"]
m_image = idata["mask"]
print("n_objects: {}\n".format(idata["n_objects"]))

with torch.no_grad():
    video, m_video = videod.synth_videos([image], [m_image])
video, m_video = video[0], m_video[0]
print("image shape: {}".format(image.shape))
print("m_image shape: {}".format(m_image.shape))
print("video shape: {}".format(video.shape)) # (T, C, H, W)
print("m_video shape: {}".format(m_video.shape))

In [ ]:
plt.imshow(image.numpy().transpose(1,2,0))
plt.figure(figsize=(18,6))

s_m_image = stack_masks(np.expand_dims(m_image.numpy(), axis=0))[0,0]
plt.imshow(s_m_image)
plt.figure(figsize=(12,6))

s_video = stack_images(video.numpy())[0].transpose(1,2,0)
plt.imshow(s_video)
plt.figure(figsize=(18,6))

s_m_video = stack_masks(m_video.numpy())[0,0]
plt.imshow(s_m_video)